In [21]:
import os
import pandas as pd
import numpy as np
import sqlite3

## Creating the pipeData table

In [23]:
# Load the CSV
TeAllPointsFile = "../data/TeAllPoints.csv"
pointsdf = pd.read_csv(TeAllPointsFile, encoding="ISO-8859-1")

# Loc column has leading spaces, so we convert to remove them
pointsdf.Loc = pd.to_numeric(pointsdf['Loc'], errors='coerce')

# pick the columns for the table
pipeDataTable = pd.DataFrame({
    'loc': pointsdf['Loc'],
    'locName': pointsdf['Loc Name'],
    'locState': pointsdf['Loc St Abbrev'],
    'locCounty': pointsdf['Loc Cnty'],
    'locZone': pointsdf['Loc Zone'],
    'dirFlo': pointsdf['Dir Flo'],
    'locType': pointsdf['Loc Type Ind']
})

pipeDataTable.head(1)

loc                    locName locState  locCounty locZone    dirFlo  \
0  70059  Chevron - Perth Amboy, NJ       NJ  MIDDLESEX       3  Delivery   

  locType  
0     UNK

## Creating the dailyFlo table

In [24]:
# Load the XLSX
flowsFile = "../data/flows_last10.xlsx"
flowsdf = pd.concat(pd.read_excel(flowsFile, sheet_name=None), ignore_index=True)

dailyFloTable = pd.DataFrame({
    'loc': flowsdf.Loc,
    'gas_date': flowsdf.Eff_Gas_Day,
    'Total_Design_Capacity': flowsdf.Total_Design_Capacity,
    'Operating_Capacity': flowsdf.Operating_Capacity,
    'Total_Scheduled_Quantity': flowsdf.Total_Scheduled_Quantity,
    'TSP_Name': flowsdf.TSP_Name
})

dailyFloTable.head(1)

loc   gas_date  Total_Design_Capacity  Operating_Capacity  \
0  70004 2020-08-21                 593830              600490   

   Total_Scheduled_Quantity      TSP_Name  
0                         0  TX EAST TRAN

## Sending data to the DB

In [2]:
conn = sqlite3.connect('ng_db_p2.db')
c = conn.cursor()

In [7]:
c.execute('''CREATE TABLE pipeData
            ([loc] INTEGER PRIMARY KEY,
            [locName] TEXT,
            [locState] TEXT,
            [locCounty] TEXT,
            [locZone] TEXT,
            [dirFlo] TEXT,
            [locType] TEXT)''')

OperationalError: table pipeData already exists

In [20]:
c.execute('''CREATE TABLE dailyFlo
          ([loc] INTEGER,
           [gas_date] date,
           [Total_Design_Capacity] INTEGER,
           [Operating_Capacity] INTEGER,
           [Total_Scheduled_Quantity] INTEGER,
           [TSP_Name] TEXT,
           PRIMARY KEY(loc,gas_date))''')

In [25]:
pipeDataTable.to_sql('pipeData', conn, if_exists='append', index = False)

In [26]:
dailyFloTable.to_sql('dailyFlo', conn, if_exists='append', index = False)

In [34]:
c.execute("SELECT * FROM pipeData")

rows = c.fetchall()

for row in rows:
    print(row)

(70004, 'DOMINION ENERGY TRANSMISSION, INC - CLARINGTON, OH', 'OH', 'MONROE', '2', 'D', 'INT')
(70011, 'COLUMBIA GAS OF  PA.  - EAGLE, PA', 'PA', 'CHESTER', '3', 'D', 'INT')
(70017, 'INDIANA GAS - GREENSBURG, IN', 'IN', 'DECATUR', '2', 'D', 'LDC')
(70018, 'INDIANA GAS - SEYMOUR, IN', 'IN', 'JACKSON', '2', 'D', 'LDC')
(70020, 'EQUITRANS - WAYNESBURG, PA', 'PA', 'GREENE', '2', 'D', 'INT')
(70030, 'PHILADELPHIA GAS WORKS - POINT BREEZE,PA', 'PA', 'PHILADELPHIA', '3', 'D', 'LDC')
(70033, 'PECO ENERGY - CROMBY GENERATION, PA', 'PA', 'CHESTER', '3', 'D', 'LDC')
(70034, 'PHILADELPHIA GAS WORKS - LEVICK ST., PA', 'PA', 'MONTGOMERY', '3', 'D', 'LDC')
(70035, 'PECO ENERGY - TILGHMAN ST. - CHESTER, PA', 'PA', 'DELAWARE', '3', 'D', 'LDC')
(70036, 'PECO ENERGY - WEST CONSHOHOCKEN, PA', 'PA', 'MONTGOMERY', '3', 'D', 'LDC')
(70039, 'UNITED CITIES - HARRISBURG, IL', 'IL', 'SALINE', '1', 'D', 'LDC')
(70040, 'Central Il. - New Dennison, IL', 'IL', 'WILLIAMSON', '1', 'D', 'UNK')
(70041, 'COLUMBIA GAS - L

In [36]:
c.execute("SELECT * FROM dailyFlo")

rows = c.fetchall()

for row in rows:
    print(row)

(70004, '2020-08-21 00:00:00', 593830, 600490, 0, 'TX EAST TRAN')
(70011, '2020-08-21 00:00:00', 433289, 444405, 124469, 'TX EAST TRAN')
(70017, '2020-08-21 00:00:00', 21573, 21573, 160, 'TX EAST TRAN')
(70018, '2020-08-21 00:00:00', 57331, 58910, 249, 'TX EAST TRAN')
(70020, '2020-08-21 00:00:00', 5011, 5488, 0, 'TX EAST TRAN')
(70030, '2020-08-21 00:00:00', 221946, 222548, 37960, 'TX EAST TRAN')
(70033, '2020-08-21 00:00:00', 123798, 119263, 0, 'TX EAST TRAN')
(70034, '2020-08-21 00:00:00', 288841, 287378, 1731, 'TX EAST TRAN')
(70035, '2020-08-21 00:00:00', 249094, 249434, 0, 'TX EAST TRAN')
(70036, '2020-08-21 00:00:00', 241034, 246546, 61291, 'TX EAST TRAN')
(70039, '2020-08-21 00:00:00', 20809, 20738, 1500, 'TX EAST TRAN')
(70040, '2020-08-21 00:00:00', 63989, 63989, 0, 'TX EAST TRAN')
(70041, '2020-08-21 00:00:00', 248781, 248781, 26707, 'TX EAST TRAN')
(70042, '2020-08-21 00:00:00', 63226, 63226, 6891, 'TX EAST TRAN')
(70043, '2020-08-21 00:00:00', 10692, 10692, 0, 'TX EAST TRA

(70059, '2020-08-24 00:00:00', 24228, 24228, 0, 'TX EAST TRAN')
(70061, '2020-08-24 00:00:00', 52353, 52353, 0, 'TX EAST TRAN')
(70062, '2020-08-24 00:00:00', 35521, 35521, 17989, 'TX EAST TRAN')
(70063, '2020-08-24 00:00:00', 10016, 10016, 1, 'TX EAST TRAN')
(70065, '2020-08-24 00:00:00', 374724, 374724, 0, 'TX EAST TRAN')
(70066, '2020-08-24 00:00:00', 145137, 145137, 6268, 'TX EAST TRAN')
(70069, '2020-08-24 00:00:00', 14434, 14434, 744, 'TX EAST TRAN')
(70070, '2020-08-24 00:00:00', 80641, 80641, 54251, 'TX EAST TRAN')
(70077, '2020-08-24 00:00:00', 300949, 300949, 0, 'TX EAST TRAN')
(70079, '2020-08-24 00:00:00', 79987, 79987, 3632, 'TX EAST TRAN')
(70081, '2020-08-24 00:00:00', 36372, 36372, 6000, 'TX EAST TRAN')
(70087, '2020-08-24 00:00:00', 1171501, 1171501, 193046, 'TX EAST TRAN')
(70089, '2020-08-24 00:00:00', 18561, 18561, 115, 'TX EAST TRAN')
(70090, '2020-08-24 00:00:00', 10922, 10922, 0, 'TX EAST TRAN')
(70092, '2020-08-24 00:00:00', 3265, 3265, 14, 'TX EAST TRAN')
(7009

(72135, '2020-08-26 00:00:00', 76202, 76202, 0, 'TX EAST TRAN')
(72137, '2020-08-26 00:00:00', 223584, 223584, 0, 'TX EAST TRAN')
(72138, '2020-08-26 00:00:00', 223806, 223806, 0, 'TX EAST TRAN')
(72158, '2020-08-26 00:00:00', 10782, 10782, 1, 'TX EAST TRAN')
(72172, '2020-08-26 00:00:00', 4903, 4903, 0, 'TX EAST TRAN')
(72185, '2020-08-26 00:00:00', 4038, 4038, 0, 'TX EAST TRAN')
(72187, '2020-08-26 00:00:00', 287695, 287695, 153107, 'TX EAST TRAN')
(72190, '2020-08-26 00:00:00', 11311, 11311, 0, 'TX EAST TRAN')
(72206, '2020-08-26 00:00:00', 33021, 33021, 33711, 'TX EAST TRAN')
(72210, '2020-08-26 00:00:00', 339584, 339584, 0, 'TX EAST TRAN')
(72222, '2020-08-26 00:00:00', 10079, 10079, 0, 'TX EAST TRAN')
(72248, '2020-08-26 00:00:00', 265102, 265102, 34639, 'TX EAST TRAN')
(72249, '2020-08-26 00:00:00', 4427, 4427, 0, 'TX EAST TRAN')
(72252, '2020-08-26 00:00:00', 16434, 16434, 0, 'TX EAST TRAN')
(72263, '2020-08-26 00:00:00', 156768, 156768, 19396, 'TX EAST TRAN')
(72266, '2020-08-

(75188, '2020-08-28 00:00:00', 434209, 434209, 0, 'TX EAST TRAN')
(75258, '2020-08-28 00:00:00', 229178, 229178, 79793, 'TX EAST TRAN')
(75275, '2020-08-28 00:00:00', 102383, 102383, 0, 'TX EAST TRAN')
(75300, '2020-08-28 00:00:00', 167545, 167545, 0, 'TX EAST TRAN')
(75301, '2020-08-28 00:00:00', 221860, 221860, 50000, 'TX EAST TRAN')
(75332, '2020-08-28 00:00:00', 755800, 755800, 279495, 'TX EAST TRAN')
(75333, '2020-08-28 00:00:00', 443872, 443872, 0, 'TX EAST TRAN')
(75475, '2020-08-28 00:00:00', 16756, 16756, 0, 'TX EAST TRAN')
(75506, '2020-08-28 00:00:00', 203542, 203542, 111460, 'TX EAST TRAN')
(75524, '2020-08-28 00:00:00', 610625, 735678, 1584, 'TX EAST TRAN')
(75525, '2020-08-28 00:00:00', 129184, 129184, 0, 'TX EAST TRAN')
(75533, '2020-08-28 00:00:00', 508500, 508500, 0, 'TX EAST TRAN')
(75553, '2020-08-28 00:00:00', 634000, 634000, 141756, 'TX EAST TRAN')
(75558, '2020-08-28 00:00:00', 500000, 500000, 9300, 'TX EAST TRAN')
(75570, '2020-08-28 00:00:00', 17124, 17124, 0, '